In [12]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11493376/11490434 [==============================] - 2s 0us/step


In [3]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128),
  tf.keras.layers.Lambda(lambda x: tf.abs(x)), 
  tf.keras.layers.Dense(10, activation='softmax')
])

In [4]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test)

Epoch 1/5
1875/1875 [==============================] - 1s 769us/step - loss: 0.2210 - accuracy: 0.9368
Epoch 2/5
1875/1875 [==============================] - 1s 766us/step - loss: 0.0886 - accuracy: 0.9733
Epoch 3/5
1875/1875 [==============================] - 1s 766us/step - loss: 0.0621 - accuracy: 0.9811
Epoch 4/5
1875/1875 [==============================] - 1s 762us/step - loss: 0.0459 - accuracy: 0.9855
Epoch 5/5
313/313 [==============================] - 0s 459us/step - loss: 0.0841 - accuracy: 0.9765


[0.08405223488807678, 0.9764999747276306]

In [5]:
def my_relu(x):
    return K.maximum(-0.1, x)

model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128),
    tf.keras.layers.Lambda(my_relu), 
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test)

Epoch 1/5
1875/1875 [==============================] - 1s 777us/step - loss: 0.2574 - accuracy: 0.9268
Epoch 2/5
1875/1875 [==============================] - 1s 785us/step - loss: 0.1137 - accuracy: 0.9665
Epoch 3/5
1875/1875 [==============================] - 1s 789us/step - loss: 0.0771 - accuracy: 0.9771
Epoch 4/5
1875/1875 [==============================] - 1s 784us/step - loss: 0.0590 - accuracy: 0.9817
Epoch 5/5
313/313 [==============================] - 0s 453us/step - loss: 0.0883 - accuracy: 0.9732


[0.08829785883426666, 0.9732000231742859]

In [6]:
from tensorflow.keras.layers import Layer

In [7]:
class SimpleLayer(Layer):
    
    def __init__(self, units = 32):
        super(SimpleLayer, self).__init__()
        self.units = units
        
    def build(self, input_shape):
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(name= 'kernel',initial_value= w_init(shape = (input_shape[-1],self.units),
                             dtype = 'float32'),
                             trainable = True)
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(name= 'bias', initial_value= b_init(shape = (self.units,), dtype= 'float32'),
                             trainable = True)
        
    def call(self, inputs):
        
        return tf.matmul(inputs, self.w)+ self.b
                

In [8]:
my_layer = SimpleLayer(units =1)

In [9]:
x = tf.ones((1,1))
y = my_layer(x)

In [10]:
print(y)

tf.Tensor([[0.07562499]], shape=(1, 1), dtype=float32)


In [13]:
# define the dataset
xs = np.array([-1.0,  0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)
ys = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0], dtype=float)

In [15]:
model= tf.keras.Sequential([my_layer])
model.compile(optimizer='sgd', loss= 'mean_squared_error')
model.fit(xs, ys, epochs = 4, verbose =0)
print(model.predict([10.0]))

[[6.5955725]]


In [16]:
print(my_layer.variables)

[<tf.Variable 'simple_layer/kernel:0' shape=(1, 1) dtype=float32, numpy=array([[0.64769495]], dtype=float32)>, <tf.Variable 'simple_layer/bias:0' shape=(1,) dtype=float32, numpy=array([0.11862279], dtype=float32)>]


### Activation custom layer

In [27]:
class MyLayerWithActivation(Layer):
    def __init__(self,units = 32, activation = None):
        super(MyLayerWithActivation, self).__init__()
        self.units = units
        self.activation = tf.keras.activations.get(activation)
        
        
    def build(self, input_shape):
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(name = 'kernel', initial_value= w_init(shape =(input_shape[-1], self.units),
                                                                    dtype = 'float32'),trainable = True)
       
        
        
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(name="bias",
            initial_value=b_init(shape=(self.units,), dtype='float32'),
            trainable=True)
        #b_init = tf.zeros_initializer()
        #self.b = tf.Variable(name = 'bias', initial_value= b_init(shape= (self.units,), dtype = 'float32',trainable = True))
        super().build(input_shape)
        
    def call(self, inputs):
        return self.activation(tf.matmul(inputs, self.w)
                              +self.b)
                             

In [28]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    MyLayerWithActivation(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test)

Epoch 1/5
1875/1875 [==============================] - 1s 791us/step - loss: 0.2969 - accuracy: 0.9134
Epoch 2/5
1875/1875 [==============================] - 1s 793us/step - loss: 0.1475 - accuracy: 0.9562
Epoch 3/5
1875/1875 [==============================] - 1s 791us/step - loss: 0.1088 - accuracy: 0.9675
Epoch 4/5
1875/1875 [==============================] - 1s 794us/step - loss: 0.0897 - accuracy: 0.9725
Epoch 5/5
313/313 [==============================] - 0s 456us/step - loss: 0.0765 - accuracy: 0.9755


[0.07646127790212631, 0.9754999876022339]

In [29]:
class SimpleQuadratic(Layer):

    def __init__(self, units=32, activation=None):
            '''Initializes the class and sets up the internal variables'''
            super(SimpleQuadratic,self).__init__()
            self.units =units
            self.activation = tf.keras.activations.get(activation)
    
    def build(self, input_shape):
        '''Create the state of the layer (weights)'''
        # a and b should be initialized with random normal, c (or the bias) with zeros.
        # remember to set these as trainable.
        a_int = tf.random_normal_initializer()
        self.a = tf.Variable(name = 'a', initial_value= a_int(shape = (input_shape[-1], self.units), dtype = 'float32'), trainable = True)
        b_int = tf.random_normal_initializer()
        self.b = tf.Variable(name = 'b', initial_value= b_int(shape = (input_shape[-1],self.units), dtype = 'float32'), trainable = True)
        c_int = tf.zeros_initializer()
        self.c = tf.Variable(name = 'c', initial_value= c_int(shape = (self.units,), dtype = 'float32'), trainable = True)
        super().build(input_shape)
   
    def call(self, inputs):
#         '''Defines the computation from inputs to outputs'''
         return self.activation(tf.matmul(tf.math.square(inputs), self.a)
                              +tf.matmul(inputs,self.b)+self.c)

In [30]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  SimpleQuadratic(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test)

Epoch 1/5
1875/1875 [==============================] - 2s 883us/step - loss: 0.2717 - accuracy: 0.9191
Epoch 2/5
1875/1875 [==============================] - 2s 892us/step - loss: 0.1291 - accuracy: 0.9606
Epoch 3/5
1875/1875 [==============================] - 2s 875us/step - loss: 0.0984 - accuracy: 0.9696
Epoch 4/5
1875/1875 [==============================] - 2s 874us/step - loss: 0.0793 - accuracy: 0.9750
Epoch 5/5
313/313 [==============================] - 0s 523us/step - loss: 0.0738 - accuracy: 0.9783


[0.07375789433717728, 0.9782999753952026]